## CrewAI

Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.

- Role-Based Agent Design: Customize agents with specific roles, goals, and tools.
- Autonomous Inter-Agent Delegation: Agents can autonomously delegate tasks and inquire amongst themselves, enhancing problem-solving efficiency.
- Flexible Task Management: Define tasks with customizable tools and assign them to agents dynamically.
- Processes Driven: Currently only supports sequential task execution and hierarchical processes, but more complex processes like consensual and autonomous are being worked on.
- Save output as file: Save the output of individual tasks as a file, so you can use it later.
- Parse output as Pydantic or Json: Parse the output of individual tasks as a Pydantic model or as a Json if you want to.
- Works with Open Source Models: Run your crew using Open AI or open source models refer to the Connect crewAI to LLMs page for details on configuring your agents connections to models, even ones running locally!

#### News Search and Summarize

Let us build an agentic system that searches for the latest news and summarizes the important points for us.

Packages needed:
* crewai
* crewai[tools]
* duckduckgo-search
* openai

In [1]:
import os
from crewai import Agent
from crewai import Task
from crewai import Crew, Process
from crewai_tools import BaseTool
from duckduckgo_search import DDGS

from dotenv import load_dotenv

load_dotenv()


True

In [2]:
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

##### Custom News Search Tool

We need to get the latest news on our topic of interest. So, let us create a 'News Search Tool' for that. A tool generally helps with a specific capability like web scraping, searching the internet, file reading etc. crewAI itself has a lot of inbuilt tools, including a search tool using Serper.

Instead of using the Serper tool, which requires an API key we will make use of DuckDuckGo search. So, let us create a custom tool for the same.

In [3]:
class NewsSearchTool(BaseTool):
    name: str = "Latest News Search tool"
    description: str = "News search tool that searches the latest news on the given topic from internet and provides the results."

    def _run(self, topic: str) -> str:
        # Using the duckduckgo python module
        ddgs = DDGS()
        # Fetching the latest news from last 1 day on a given topic
        results = ddgs.news(keywords=topic, timelimit="d", max_results=20)
        result_string=""
        # Combine the news topic and description
        for result in results:
            result_string += f"Title: {result['title']} \n Description: {result['body']}\n\n"
        # In case if there are no news
        if len(result_string.strip()) < 2:
            result_string="No news found for the given keyword"
        return result_string.strip()

news_search_tool = NewsSearchTool()


Now let us create agents for our task. We will create two agents:

- **Reasearcher agent:** This LLM agent will use the news search tool to research about a specific topic and provide the results.
- **Summarizer agent:** This LLM agent will summarize the news provided by the researcher agent.

In crewAI, we can use the Agent to create the agents. We need to provide the role, goal, backstory and tools to the agent.

In [4]:
# Creating a researcher agent with memory and verbose mode
researcher = Agent(
  role='Researcher',
  goal='Get the latest news in {topic}',
  verbose=True,
  memory=True,
  backstory=(
      "As a knowledge enthusiast, you always look for new happenings on a given topic "
      "and share the knowledge with broader community."
  ),
  tools=[news_search_tool],
)


In [5]:
# Creating a summarizer agent with custom tools and delegation capability
summarizer = Agent(
  role='Summarizer',
  goal='Summarize the latest news in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you "
    "remove redundant information and summarize the "
    "discoveries in an accessible manner."
  ),
  allow_delegation=False
)


Now we need to specify the tasks that each of these agents need to perform. We will have two tasks

- Research Task: This task uses the news search tool to get the latest news.
- Summarize Task: This task summarizes the news into 2 or 3 bullet points.

In [6]:
# Research task
research_task = Task(
    description=(
    "Get the latest news in {topic}."
    ),
    expected_output='Points covering the latest news on {topic}.',
    tools=[news_search_tool],
    agent=researcher,
)

# Summarizer task 
summarize_task = Task(
    description=(
      "Given all the news about the {topic}, summarize them to 2 or 3 bullet points."
      "Discard the news which do not have central focus on {topic}."
      "If no news is relevant, return that no news found for the {topic}"
    ),
    expected_output=(
    "2 or 3 summarized bullet points, each one focussing on one specific news "
    "about the {topic}. If no news found, return no news found"
    ),
    agent=summarizer,
    output_file='news_summary.md',  # Example of output customization
)


Now let us form the crew (agentic system) using the created agents and tasks, and start the process.

In [7]:
# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, summarizer],
  tasks=[research_task, summarize_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
)

In [8]:
result = crew.kickoff(inputs={'topic': 'HDFC Bank'})
print(result)



> Entering new CrewAgentExecutor chain...
I need to use the Latest News Search tool to gather the latest news on HDFC Bank.

Action:
Latest News Search tool

Action Input:
{"topic": "HDFC Bank"}
 

Title: HDFC Bank top executives highest paid bankers in FY24, ICICI brass next 
 Description: Sashidhar Jagdishan, managing director (MD) and chief executive officer (CEO) of HDFC Bank, drew a remuneration of Rs 10.77 crore in FY24, according to the lender's annual report. Kaizad Bharucha, deputy MD at HDFC Bank, drew a remuneration of Rs 11.12 crore.

Title: HDFC Bank Share Price Live Updates: HDFC Bank Sees Slight Increase in Price, Stable 3-Year Returns Amid Market Volatility 
 Description: Join us on the HDFC Bank Stock Liveblog, your hub for real-time updates and comprehensive analysis on a prominent stock. Stay in the know with the latest information about HDFC Bank, including: Last traded price 1656.

Title: Alert for HDFC Bank Customers; These Services to be Down, Check Maintenance

In [9]:
print(result)

- HDFC Bank's top executives, including the MD and CEO, had the highest remuneration in FY24.
- HDFC Bank's share price saw a slight increase and stable 3-year returns amid market volatility.
- HDFC Bank customers were alerted about scheduled maintenance for digital platforms.
